In [1]:
import numpy as np
import queue
import copy
import matplotlib.pyplot as plt


In [9]:
# unit: hour
#即核酸检测有两个小时的检测窗口 eg. 9-11 a.m. 
#假设所有人都在这个区间进行检测👉后续可以优化
# simulation(hours)
#arrival rate (/hour) i.e. 2 per min
#service rate (/hour) 这里考虑改进前 即50s左右
#我们假设在正式检测开始之前，已经有人开始排队👉即这里的‘begin', 是一个0-30之间的随机整数
#number of server

begin = np.random.randint(0,30)
c = 1
total_time = 2
IAT_rate = 120
ST_rate = 72*c
rho = IAT_rate/ST_rate
print(begin)

18


In [10]:
# Initialize Parameters
# IAT  empty array for putting random number inter-arrival time
# St   ... for serving time rate
# AT: arrival times
# ST: serve time


qu = queue.Queue()
curr_process = None
IAT = []
ST = []
AT = []
wait_time = []
server_busy = False
list_wait = []
list_delay = []


In [11]:
# num_processes 即一共有多少个人在检测窗口里来做核酸检测

num_processes = int(np.random.poisson(IAT_rate)* total_time) 
total_people = num_processes + begin
num_processes_served = 0
print(num_processes)
print(begin)
print(total_people)

240
18
258


In [13]:
IAT_new = []

In [14]:
#Populate new Inter-Arrival-Times (IAT_new)
#即在正式检测开始之前到达的人的inter arrival time=0

for i in range(total_people):
    temp = np.random.exponential(1/IAT_rate)*60*60
    if i<begin:
        IAT_new.append(0)
        i=i+1
    else: 
        IAT_new.append(int(temp-temp%1))

print(IAT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 15, 23, 20, 16, 8, 48, 13, 4, 4, 64, 17, 11, 16, 28, 19, 77, 5, 36, 47, 90, 15, 57, 45, 39, 39, 37, 47, 49, 22, 33, 57, 34, 2, 22, 20, 6, 3, 22, 18, 56, 14, 28, 2, 6, 12, 12, 7, 40, 34, 12, 54, 24, 13, 4, 9, 6, 1, 40, 15, 56, 56, 83, 34, 3, 6, 78, 3, 3, 49, 40, 6, 115, 24, 55, 98, 9, 28, 31, 38, 119, 1, 39, 132, 8, 54, 30, 109, 7, 94, 17, 26, 33, 101, 5, 0, 32, 61, 6, 2, 100, 4, 19, 29, 15, 21, 60, 6, 9, 80, 17, 86, 4, 0, 32, 11, 43, 138, 7, 107, 67, 17, 16, 48, 43, 15, 70, 1, 39, 5, 15, 69, 24, 97, 10, 15, 59, 41, 1, 4, 2, 4, 55, 26, 12, 0, 6, 60, 4, 43, 47, 36, 21, 1, 40, 26, 4, 26, 7, 11, 56, 12, 6, 7, 41, 41, 21, 3, 12, 28, 84, 20, 19, 60, 5, 34, 25, 26, 39, 8, 6, 73, 0, 16, 21, 81, 20, 13, 49, 3, 16, 70, 94, 29, 38, 106, 64, 9, 65, 0, 6, 11, 2, 18, 7, 62, 38, 108, 19, 79, 29, 60, 55, 36, 19, 19, 2, 1, 27, 1, 3, 4, 30, 17, 7, 5, 16, 41, 86, 6, 25, 0, 3, 15, 0, 94, 143, 1, 31, 4]


In [15]:
# Populate Service-Times (ST) (where ST[i]!=0)

while not len(ST) == total_people:
    temp = np.random.exponential(1/ST_rate)*60*60
    if not int(temp- temp%1)<1:
        ST.append(int(temp - temp%1))

print(ST)

[41, 20, 1, 53, 6, 106, 35, 5, 40, 5, 20, 20, 26, 49, 155, 102, 98, 43, 35, 82, 17, 53, 83, 126, 18, 112, 25, 11, 30, 85, 26, 85, 187, 27, 27, 37, 20, 46, 50, 29, 39, 67, 25, 280, 1, 92, 10, 111, 23, 3, 207, 18, 12, 28, 58, 31, 23, 13, 86, 33, 56, 67, 76, 114, 68, 77, 34, 90, 3, 61, 13, 63, 43, 12, 14, 114, 1, 56, 39, 68, 330, 35, 31, 1, 27, 150, 57, 103, 46, 23, 112, 52, 21, 155, 43, 167, 112, 14, 26, 16, 39, 39, 12, 158, 29, 108, 27, 49, 119, 89, 54, 15, 69, 60, 65, 42, 10, 29, 9, 81, 44, 5, 92, 91, 38, 46, 27, 26, 35, 15, 32, 15, 37, 58, 47, 265, 7, 15, 75, 80, 30, 31, 7, 214, 26, 39, 9, 3, 178, 17, 2, 83, 80, 123, 99, 34, 104, 84, 76, 4, 5, 46, 143, 174, 71, 100, 52, 38, 46, 34, 39, 135, 18, 76, 15, 34, 30, 29, 67, 101, 110, 268, 10, 26, 3, 11, 40, 22, 101, 23, 80, 26, 30, 75, 4, 155, 4, 77, 261, 21, 117, 27, 12, 18, 68, 38, 83, 75, 1, 166, 37, 173, 36, 12, 48, 111, 6, 8, 69, 69, 44, 10, 9, 77, 36, 141, 93, 3, 27, 28, 3, 27, 8, 101, 67, 146, 44, 120, 16, 18, 40, 120, 43, 34, 2, 106

In [16]:
# save a copy of ST

ST_copy = copy.deepcopy(ST)

In [17]:
AT_new = []

In [18]:
# Get new Arrival-Times (AT) from IAT starting at t=0 
# 生成到达时间队列
# and initialize Waiting-Times to 0

for i in range(total_people):
    if i<begin:
        AT_new.append(0)
        i=i+1
    else:
        AT_new.append(IAT_new[i]+AT_new[i-1])
    wait_time.append(0)

print(AT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 21, 44, 64, 80, 88, 136, 149, 153, 157, 221, 238, 249, 265, 293, 312, 389, 394, 430, 477, 567, 582, 639, 684, 723, 762, 799, 846, 895, 917, 950, 1007, 1041, 1043, 1065, 1085, 1091, 1094, 1116, 1134, 1190, 1204, 1232, 1234, 1240, 1252, 1264, 1271, 1311, 1345, 1357, 1411, 1435, 1448, 1452, 1461, 1467, 1468, 1508, 1523, 1579, 1635, 1718, 1752, 1755, 1761, 1839, 1842, 1845, 1894, 1934, 1940, 2055, 2079, 2134, 2232, 2241, 2269, 2300, 2338, 2457, 2458, 2497, 2629, 2637, 2691, 2721, 2830, 2837, 2931, 2948, 2974, 3007, 3108, 3113, 3113, 3145, 3206, 3212, 3214, 3314, 3318, 3337, 3366, 3381, 3402, 3462, 3468, 3477, 3557, 3574, 3660, 3664, 3664, 3696, 3707, 3750, 3888, 3895, 4002, 4069, 4086, 4102, 4150, 4193, 4208, 4278, 4279, 4318, 4323, 4338, 4407, 4431, 4528, 4538, 4553, 4612, 4653, 4654, 4658, 4660, 4664, 4719, 4745, 4757, 4757, 4763, 4823, 4827, 4870, 4917, 4953, 4974, 4975, 5015, 5041, 5045, 5071, 5078, 5089, 5145, 5157, 5163, 5170,

In [19]:
# simulate M/M/1 queue

for i in range(total_time*60*60):    
    if server_busy:
        for item in list(qu.queue):
            wait_time[item] = wait_time[item] + 1
        ST[curr_process] = ST[curr_process] - 1
        if ST[curr_process] == 0:
            server_busy = False
            num_processes_served = num_processes_served + 1

    for j in range(total_people):
        if i== AT_new[j]:
            qu.put(j)

    if not server_busy and not qu.empty():
        curr_process = qu.get()
        server_busy = True
        


In [20]:
# 总共等待时间
# 总共delay时间 = total time in the system 
# delay = waiting time + serve time
sum_wait = 0
sum_delay = 0

In [21]:
print(num_processes_served)

125


In [22]:
for i in range(num_processes_served):
    sum_wait = sum_wait + wait_time[i]
    sum_delay = sum_delay + wait_time[i] + ST_copy[i]
    
if num_processes_served == 0:
    list_wait.append(0)
    list_delay.append(0)
else:
    list_wait.append(sum_wait/(num_processes_served*60*60))  
    list_delay.append(sum_delay/(num_processes_served*60*60))


In [23]:
print(total_people)
print(begin)
print(num_processes)
print(num_processes_served)

258
18
240
125


In [24]:
#平均等待时间 in hour
print(sum_wait/(num_processes_served*60*60))
print(sum_delay/(num_processes_served*60*60))

# 平均等待时间 in minute
print(sum_wait/(num_processes_served*60))

0.5767533333333333
0.5926622222222222
34.6052


In [23]:
print((6000/worktime)*2)
print(salary/worktime)

65.21739130434783
50.14673913043478
